In [ ]:
import json
import sys
import os
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
sns.set(style="whitegrid")
%matplotlib inline

# Milestone 2; 10,000 battles Project

## Tasks Description

Milestone 2 (20%): the project repo contains a notebook with data collection and descriptive analysis, properly commented, and the notebook ends with a more structured and informed plan for what comes next.

The tasks involving a large amount of data were pre-run and we simply describe their purposes and outputs while showing how to call them in comments.

## Data Collection

Data collection was a significant part of our work, given the nature of our original dataset. The task was to go from a 44 Gb large wikipedia dump to clean and normalized features about each battle. As it was shown during the lecture, data collection is in fact an iterative process: new needs in the analysis part may require new data to be extracted, or different transformation applied. Thus, the data collection was organized into a **pipeline** of 3 operations, in order to achieve **composability** and **reproducibility**. We know explain this pipeline as a part of this notebook, the actual code being organized as **python modules**, much more suited for data processing than notebooks.

Each step of the pipeline is a **python script** in the `processing` folder. They have the following usage:

```shell
python script.py file-name-in file-name-out
```

For the sake of reproducibilty and organization, we used a naming convention for the output files including what step of the pipline was run, and the version of this dataset. Each version of the dataset is then associated with a git tag marking the state of the codebase that generated the file, to avoid confusion when coming back to the very begining of the pipeline each time we have a doubt (see README in the `datasets/` folder). We know describe each of the 3 pipeline operation.

### Page extraction

**Script**: `page_extraction.py`<br />
**Environmnent**: Cluster<br />
**Input**: `hdfs:///datasets/wikipedia/enwiki-latest-pages-articles-multistream.xml` (~44 Gb)<br />
**Output**: `battle-pages-v.json` (~123 Mb)<br />
**Description**:<br />
Pages extraction has two main goals, it selects what pages are (entierly) kept in the next step using a regular expression in the title (may be refined later, if we find a better way to isolate a battle related page), and translate from an XML to a JSON representation for easier python processing. It leverages on [pySpark's DataFrame](https://docs.databricks.com/spark/latest/dataframes-datasets/introduction-to-dataframes-python.html) and its custom [XML data-source from Databricks](https://github.com/databricks/spark-xml) to provide an SQL-like, parallelized Spark job.


### Fields extraction

**Script**: `fields_extraction.py`<br>
**Envir.**: Local<br>
**Output**: `battle-fields-v.json` (~14 Mb)<br>
**Description**:<br>
This step's purpose is to extract key-value pairs from the raw page [Wikitext](https://www.mediawiki.org/wiki/Wikitext), where keys are identifiers of information contained in the page and values are either a `dict` of other key-values pairs, a `list` of string, or a string (mainly, strings that are actually wikitext). In other words, we parse the page into a tree-like structure on which it will be easier to parse and extract actual features afterward. We mainly (but not exclusively) relied on the presence of an `infobox` [template](https://en.wikipedia.org/wiki/Help:Template) for most of the battle pages. Again, each line contains the tree of a battle. Therefore:

In [ ]:
# We can load all the battle trees like this
battles = [json.loads(line) for line in open('datasets/battle-fields-1.json')]
print("Number of pages or battles pages", len(battles))

We then perform some preliminary assessments about the current state of the dataset.

In [ ]:
errors = [b.get('infobox').get('error') for b in battles]
no_infoboxes = sum([1 for e in errors if e=="no infobox"])
more_infoboxes = sum([1 for e in errors if e=="more than one infobox"])

print("Number of pages that do not contain an infobox ", no_infoboxes)
print("Number of pages that contains more than one infobox ", more_infoboxes)
print("Number of pages that do contain an infobox ", len(battles)-no_infoboxes-more_infoboxes)

We immediately see that the number of page we can extract information from is greately reduced. It later came out that many of the pages in the dump are alias, redirects and discussion pages. However, the new battle count is still comfortably high to provide interesting analysis.

We then assess the actual population of the extractable keys, so that we know on what field we can focus our feature extraction effort:

In [ ]:
df = pd.DataFrame([b["infobox"] for b in battles if not b["infobox"].get("error")])
f, ax = plt.subplots(figsize=(15, 20))
counts = df.count().sort_values(ascending=False)
counts = counts[counts > 20]
sns.barplot(x=counts, y=counts.index, ax=ax)
plt.show()

Based on these observations, we selected the following set of features to be extracted from the key-value pairs by the next pipeline step (we may add more of them later):
- date
- coordinates
- combatants (combatant1, ...)
- result
- strengths (strength1, ...) in terms of number of men
- casualties (casualties1, ...) in terms of number of men killed/wounded/captured

### Features extraction

**Script**: `features_extraction.py`<br />
**Envir.**: Local<br />
**Output**: `battle-features-v.json` (~4.1 Mb)
**Description**:<br>
The final step extract a "flat" set of key-value from the preivious step's tree (e.i., values cannot be another `dict`) that we call features. It does so by first further parsing and transforming values from the previous step into a normalized representation, possibly combining multiple ones togheter. This was indeed the most time-consuming part of the current implementation, as it involves parsing sometime highly variable, unormalized free-text data. We elaborate on this aspect for some features in the following subsections. Each of line in the output file contains such set, so it can be convieniently imported in a `pandas.DataFrame` object.

In [ ]:
battles = pd.DataFrame([json.loads(line) for line in open("datasets/battle-features-0.json")])
print(battles.count())
demo_col = ["combatant_first_1", "combatant_first_2", "result_combatant_1", "result_combatant_2", "start_date", "end_date", "casualties_1", "casualties_2"]
battles[demo_col].head()

## Descriptive analysis

We provide a succint descriptive analysis for the main features. 

### Geolocation

We begin by showing the geographical coverage of our dataset. This seems to be consistent with the location of wars across history.

In [ ]:
from folium.plugins import HeatMap
import folium as fl
m = fl.Map()
coord_df = battles[["latitude", "longitude"]].dropna()
coords = [[lat, long] for lat, long in zip(coord_df["latitude"], coord_df["longitude"])]
HeatMap(coords).add_to(m)
m

### Dates

We continue by looking at the time domain coverage. It looks like we have a pretty uniform one.

In [ ]:
import datetime
dates = battles[battles.start_date.notna()].start_date
date_bc = battles[battles.start_date.notna()].dates_bc

X=[datetime.datetime.strptime(date, "%Y-%m-%d") for date, bc in zip(dates, date_bc) if not bc]
fig, ax = plt.subplots(figsize=(20,1))
ax.scatter(X, [1]*len(X),
           marker='|', s=100)

ax.yaxis.set_visible(False)
ax.spines['right'].set_visible(False)
ax.spines['left'].set_visible(False)
ax.spines['top'].set_visible(False)
ax.xaxis.set_ticks_position('bottom')

ax.get_yaxis().set_ticklabels([])
day = pd.to_timedelta("1", unit='D')
plt.xlim(X[0] - day, X[-1] + day)
plt.show()

We group here all our functions to plot:

In [ ]:
def densplot(columns, xlabel, title, axo):
    for i,v in enumerate(columns):
        sns.distplot(v, ax=axo, kde_kws={"label": i})
    axo.set_title(title)
    axo.set_xlabel(xlabel, fontsize=12)
    
def scatplot(xelem, yelem, xlabel, ylabel, title, axo, polyfit=None):
    axo.scatter(xelem, yelem)
    if polyfit:
        plt.plot(np.unique(xelem), np.poly1d(np.polyfit(xelem, yelem, polyfit))(np.unique(xelem)), 'C2')
    axo.set_title(title)
    axo.set_xlabel(xlabel, fontsize=12)
    axo.set_ylabel(ylabel, fontsize=12)

In [ ]:
battles = [json.loads(line) for line in open("datasets/battle-features-0.json")]
df = pd.DataFrame(battles).replace(0, np.NaN)

#### Casualties

A battle has at maximum casualties for 4 different combatants ("casualties_1", "casualties_2", ...). And each casualties information can contain a number of people "killed", "wounded", "missing" or "captured". When possible, we also fetched (parsed) these data. We have observed that for multiple battles, wikipedia was referencing multiple sources for the same number and we handled this by doing the average of the values. We also did the average for range values. 

In [ ]:

df[['killed_1', 'wounded_1', 'missing_1', 'captured_1', 'casualties_1']].head()

In [ ]:
df[['killed_1', 'wounded_1', 'missing_1', 'captured_1', 'casualties_1', 'killed_2', 'wounded_2', 'missing_2', 'captured_2', 'casualties_2']].count()

We observe that the casualties is most of the time defined by the number of killed and wounded people.
We focus on the total number of casualties for this descriptive analysis and will go into more details in the future exploratory analysis.

We observe that the dataset contains a lot of NaN values. This is mainly because there is no numeric information for the casualties. In our data extraction pipeline we have observed that casualties_1 and casualties_2 contain only 79 and 80 lines (battles) that have a numeric information but cannot be parsed.  

In [ ]:
sums_null = df[['casualties_1', 'casualties_2', 'casualties_3', 'casualties_4']].isnull().sum()
sums_non_null_percent = 100-(100*sums_null/len(df))
print("number of null values")
print(sums_null)
print("number of non null values")
print(len(df)-sums_null)
plt.bar(range(len(sums_null)), sums_non_null_percent)
plt.title("Percent of battles with numeric values for casualties")
plt.xticks(range(len(sums_null)), sums_null.keys())
plt.show()

We also observe that almost 60% of the battles have numeric values for two combatants ("casualties_1" and "casualties_2"). We also notice that casualties_1 and 2 have more data points than the other. This makes sense as we can usually consider two opposite sides in a battle.
Each casualties feature has an average of:

In [ ]:
averages = df[['casualties_1', 'casualties_2', 'casualties_3', 'casualties_4']].mean()
print(averages)

We can see that the average for 1 and 3 are close, while casualties_2 is higher. This may indicate that usually the combatant 2 (to which corresponds casualties_2) is usually the looser of the battle. Even though it is too early to jump to conclusions !!
In the following, we observe the distributions of the features. Since casualties_4 only has 5 data points, we do not use it in our analysis.

In [ ]:
fig, ((ax1, ax2), (ax3, ax4), (ax5, ax6)) = plt.subplots(nrows=3, ncols=2, sharey=False, figsize=(15,20))
densplot([df['casualties_1'].dropna()], 'casualties_1', "casualties_1 density", ax1)
densplot([df.query('2000 > casualties_1 >1')['casualties_1']], 'casualties_1', "casualties_1 density (ZOOM) ", ax2)

densplot([df['casualties_2'].dropna()], 'casualties_2', "casualties_2 density", ax3)
densplot([df.query('2000 > casualties_2 >1')['casualties_2']], 'casualties_2', "casualties_2 density (ZOOM) ", ax4)

densplot([df['casualties_3'].dropna()], 'casualties_3', "casualties_3 density", ax5)
densplot([df.query('2000 > casualties_3 >1')['casualties_3']], 'casualties_3', "casualties_3 density (ZOOM) ", ax6)

fig.tight_layout()
plt.show()

We observe that casualties 1,2 and 3 are all pretty sparsed in their values while they all have there peak for small casualties values. By "zooming", we observe that most of the values are between 0 and 2000 casualties. For casualties_4 which contain much less data points, we observe that the values are between 0 and 2000 too.

For the remaining of our analysis for this feature we will focus on casualties_1 (c1) and casualties_2 (c2).
We first observe that 3782 out of the 4392 (min(#casualties_1, #casualties_2)) battles have information on two combatants' casualties.

In [ ]:
print("number of battles with values for c1 and c2: ", (len(df.query('0 < casualties_1 and 0 < casualties_2')['casualties_1'])))

As a first step towards our future exploratory analysis, we can combine these two features, to see if, for example, high casualties for 1 combatant, means also high casualties for the other.

In [ ]:
c1_zoom = df.query('25000 > casualties_1 >1 and 25000 > casualties_2 > 1')['casualties_1']
c2_zoom = df.query('25000 > casualties_1 >1 and 25000 > casualties_2 > 1')['casualties_2']

c1 = df.query('casualties_1 >1 and casualties_2>1')['casualties_1']
c2 = df.query('casualties_1 >1 and casualties_2>1')['casualties_2']


fig, ((ax1, ax2), (ax3, ax4)) = plt.subplots(nrows=2, ncols=2, sharey=False, figsize=(10,10))
scatplot(c1, c2, 'casualities_1', 'casualities_2', "Casualities 1 vs. 2", ax1)
scatplot(c1_zoom, c2_zoom, 'casualities_1', 'casualities_2', "Casualities 1 vs. 2 (ZOOM)", ax2)
densplot([abs(c1-c2)], 'c1', "Difference between c1 and c2 in density", ax3)
densplot([abs(c1_zoom-c2_zoom)], 'c1', "Difference between c1 and c2 in density (ZOOM)", ax4)
ax4.set_xlim(0,10000)
fig.tight_layout()
plt.show()

We observe that the difference between the casualties is concentrated to 0. We observe that we cannot conclude that high casualties on one side does not mean high casualties for the other.

#### Strengths

Strength is the number of men involved in a battle. A battle has at maximum strength information for 3 different combatants ("strength_1", "strength_2", ...). We also did the average for range values. 

In [ ]:
df[['strength_1', 'strength_2', 'strength_3']].head()

We observe that the dataset contains a lot of NaN values. This is mainly because there is no numeric information for the strength. In our data extraction pipeline we have observed that strength_1 and strength_2 contain only 120 and 110 lines (battles) that have a numeric information but cannot be parsed. (this cans also be the case that it is numeric information but irrelevant, so it is not parsed on purpose.

In [ ]:
sums_null = df[['strength_1', 'strength_2', 'strength_3']].isnull().sum()
sums_non_null_percent = 100-(100*sums_null/len(df))
print("number of null values")
print(sums_null)
print("number of non null values")
print(len(df)-sums_null)
plt.bar(range(len(sums_null)), sums_non_null_percent)
plt.title("Percent of battles with numeric values for casualties")
plt.xticks(range(len(sums_null)), sums_null.keys())
plt.show()

We also observe that almost 70% of the battles have numeric values for two combatants ("strength_1" and "strength_2"). We also notice that strength_1 and 2 have more data points than the other. Again, this makes sense as we can usually consider two opposite sides in a battle.
Each casualties feature has an average of:

In [ ]:
averages_strength = df[['strength_1', 'strength_2', 'strength_3']].mean()
print(averages_strength)

We can see that the average for the 3 features is pretty similar. This may indicate that usually the battles oppose two sides that have a similar number of fighters.
We observe the distributions of the features: (Again we will focus on the first two as the last one only has 14 data points)

In [ ]:
fig, ((ax1, ax2), (ax3, ax4)) = plt.subplots(nrows=2, ncols=2, sharey=False, figsize=(15,10))
densplot([df['strength_1'].dropna()], 'strength_1', "strength_1 density", ax1)
densplot([df.query('2000 > strength_1 >1')['strength_1']], 'strength_1', "strength_1 density (ZOOM) ", ax2)

densplot([df['strength_2'].dropna()], 'strength_2', "strength_2 density", ax3)
densplot([df.query('2000 > strength_2 >1')['strength_2']], 'strength_2', "strength_2 density (ZOOM) ", ax4)

fig.tight_layout()
plt.show()

We observe similar results as for casualties, in fact strength 1,2 and 3 are all pretty sparsed in their values while they all have there peak for small strength values. By "zooming", we observe that most of the values are between 0 and 2000 for strength.

For the remaining of our analysis for this feature we will focus on strength_1 (s1) and strength_2 (s2).
We first observe that 4383 out of the 5104 (min(#strength_1, #strength_2)) battles have information on two combatants' strengths.

In [ ]:
print("number of battles with values for s1 and s2: ", (len(df.query('1 < strength_1 and strength_2 > 1')['strength_1'])))

We now combine these two features, to see if, for example, high strength for 1 combatant, means also high strength for the other, as the average and distribution results tend to show.

In [ ]:
s1_zoom = df.query('25000 > strength_1 >1 and 25000 > strength_2 > 1')['strength_1']
s2_zoom = df.query('25000 > strength_1 >1 and 25000 > strength_2 > 1')['strength_2']

s1 = df.query('strength_1 >1 and strength_2>1')['strength_1']
s2 = df.query('strength_1 >1 and strength_2>1')['strength_2']


fig, ((ax1, ax2), (ax3, ax4)) = plt.subplots(nrows=2, ncols=2, sharey=False, figsize=(13,10))
scatplot(s1, s2, 'strength_1', 'strength_2', "Strengths 1 vs. 2", ax1)
scatplot(s1_zoom, s2_zoom, 'strength_1', 'strength_2', "strength 1 vs. 2 (ZOOM)", ax2)
densplot([abs(s1-s2)], 's1', "Difference between s1 and s2 in density", ax3)
densplot([abs(s1_zoom-s2_zoom)], 's1', "Difference between s1 and s2 in density (ZOOM)", ax4)
ax4.set_xlim(0,10000)
fig.tight_layout()
plt.show()

These first results tend to show that usually the battles oppose two sides with a close strength. Indeed, the difference between the two strength is usually small. We also observe that, the number seems to be pretty often round up in wikipedia by looking at the top right graph where we can see kind of grid patterns.

#### Combatants

A battle has at maximum 3 different combatants ("combatant_1", "combatant_2", ...). We have noticed that wikipedia often contains multiple combatants in one combatant feature. For examples, during the world war, the battles usually opposed two sides while each side was made of an alliance of multiple combatants. Thus, we retrieve the main combatant and a list of all the combatants present for each battle for each feature.

In [ ]:
df[['combatant_first_1', 'combatant_list_1', 'combatant_first_2', 'combatant_list_2']].head()

In [ ]:
sums_null = df[['combatant_first_1', 'combatant_first_2', 'combatant_first_3']].isnull().sum()
sums_non_null_percent = 100-(100*sums_null/len(df))
print("number of null values")
print(sums_null)
print("number of non null values")
print(len(df)-sums_null)
plt.bar(range(len(sums_null)), sums_non_null_percent)
plt.title("Percent of battles with numeric values for casualties")
plt.xticks(range(len(sums_null)), sums_null.keys())
plt.show()

We observe that in contrary with the previous features, most of the battles contain information about the combatants. Again most of the battles are between two (group of) combatants.

Since the number of different combatants is pretty high: 4719. We will show here the 50 combatants that participated in the higher number of battles. An not surprisingly (!), the U.S. are first, right before France and Spain.

In [ ]:
cbt_1 = np.array(df["combatant_list_1"].dropna())
cbt_2 = np.array(df["combatant_list_2"].dropna())
cbt_3 = np.array(df["combatant_list_3"].dropna())
print(len(cbt_1), len(cbt_2), len(cbt_3))

cbt_all = np.concatenate((cbt_1, cbt_2, cbt_3))

all_cbt_names = [c for cl in cbt_all for c in cl]
names = pd.Series(all_cbt_names)
print(len(set(all_cbt_names)))

f, ax = plt.subplots(figsize=(6, 15))
counts = names.value_counts().sort_values(ascending=False)
counts = counts.head(50)
sns.barplot(x=counts, y=counts.index, ax=ax)
counts

#### Results

When parsing the battles' results, we mapped them to a qualifier and a result type.
A qualifier can be: "decisive", "major", "crushing", "tactical" or "strategic", while the result type is in: "victory", "defeat", "retreat".
The results correspond then to each combatant (or group of combatant).

In [ ]:
df[['result_combatant_1','result_combatant_2','result_combatant_3', 'indecisive', 'tactical_indecisive', 'strategic_indecisive']].head()

In [ ]:
fig, ((ax1, ax2)) = plt.subplots(nrows=1, ncols=2, sharey=False, figsize=(13,5))
counts1 = df['result_combatant_1'].value_counts().sort_values(ascending=False)
counts1 = counts1.head(6)
sns.barplot(x=counts1, y=counts1.index, ax=ax1)
counts2 = df['result_combatant_2'].value_counts().sort_values(ascending=False)
counts2 = counts2.head(6)
sns.barplot(x=counts2, y=counts2.index, ax=ax2)
fig.tight_layout()

We observe that for each feature, the number of empty values (denoted by empty string) is high (about 45% of the dataset for combatant_1).
We also observe that the most present type is victory, while the qualifier of the victory is precised in some cases.

We provide here the new version of our readme:

# 10,000 Battles

## Abstract
In this project we will better understand one ubiquitous component of war: the battle. We propose to do so by retreiving and studying public data provided by Wikipedia about battles located around the world and which happened through the centuries of human history. By combining theses comprehensive battle records, we aim to observe how they evolved in terms of duration, localisation, geography, size, casualties through some exploratory analysis. Moreover, we will extract information on the belligerent forces and victory types (strategic, tactic, ...) for each battle. Our goal is then to find relations between such information and other variables. Leveraging on visualization techniques, we plan to gain knowledge on modern warfare from a data perspective and present these findings in an intuitive way. Our intent is to answer interesting questions such as: is there a repetitive pattern in the history of battles or wars in general? Are the conflicts becoming more deadly or less frequent?...

## Research questions
We intend to determine:
- how the duration of battles changed over time,
- how the battles are spread geographically over time,
- if there is a repetitive pattern in the history of battles,
- how the number of casualties is evolving,
- if we can relate battle-related variables to the victory type.


## Datasets

### Main
#### Milestone 1
We will mainly rely on **en.wikipedia.org** to gather information and statistics about battles around the globe. We have estimated there are around 15,000 wikipedia pages dedicated to distinct battles and not including redirect pages. Random exploration showed that a vast majority of them contain a summary table including dates, locations, result, belligerents, commanders, units and losses. Therefore, we should have between 7000 to 12000 pages containing meaningful and extractable data. One key information provided by this dataset will be the victory type (tactical, strategic, decisive, pyrrhic,...), to which we propose to pay special attention. This may require some interesting natural language processing techniques to exploit, but would also be very interesting to analyse. In the same idea, some battle are specifically labeled as resulting in a territory expansion, another interesting characteristic.
#### Milestone 2
**We have observed that we were able to extract more than 8,000 wikipedia pages about distinct battles. During the retrieving and parsing of the data, we have observed that around 70% of the data were parseable. We have managed to extract and parse the features that we consider interesting for our project, namely the battle's title, date, geographical coordinates, combatants, strength (number of fighters) and casualties.**

### Enrichment
#### Milestone 1
Preliminary random exploration revealed that the more recent is the battle, the more difficult it is to parse the wikipedia table. This is due to the increase of detailed information (mainly about casualities). For this recent part of history we can leverage on the UCDP Battle-Related Deaths Dataset to extract casualties data. This second dataset contains events at a much smaller granularity. It can be used to compare battles in the wider context of "violent Events".
#### Milestone 2
**We have observed that we were able to correctly and efficiently parse about 70% of the data in wikipedia. Even though this part took us more time than expected, we achieved satisfying results and decided to use these information to continue our project. In fact, we consider that we have enough information to compute interesting and meaningful results in order to answer our research questions. Thus, we do not need the UCDP Battle-Related Deaths Dataset.**

To this point of our analysis, all datasets and computations we are considering can be handled by a simple laptop or server. It might however be convenient for us to perform the data extraction from the Wikipedia dataset on the cluster (or at least the page extraction), so we don't have to host large file on our machines.
**We used the cluster and spark jobs to extract battle information from the wikipedia dump. The other operations, such as each battle feature extraction and the parsing of the data can be done locally on a laptop.**

## Roadmap for Milestone 2
Oct. 31, 2017: First version of the readme

Nov. 10, 2017: First clear and practical representation of the available data.

Nov. 24, 2017: Exploratory analysis of the data and description of interesting results.

Nov. 28, 2017: Plan for the next steps.

Nov. 28, 2017 (23:59 CET): milestone 2 (20%): the project repo contains a notebook with data collection and descriptive analysis, properly commented, and the notebook ends with a more structured and informed plan for what comes next.

## Roadmap towards Milestone 3
**Dec. 7, 2017: Exploratory Analysis: after the descriptive analysis of the dataset we constructed from available data, we will do a proper and complete exploratory analysis of the data. This one will be done in multiple steps:**
**- We will create a list of hypothesis, summarizing the correlations and other relations that we will investigate and that we need to answer our resarch questions**
**- We will do the exploratory analysis. In which we will focus on our research questions while still not forgetting to check for other relations that can be interesting.**
**- We will study some related work such as https://ourworldindata.org/war-and-peace/ and https://www.youtube.com/watch?v=DwKPFT-RioU.**
**- We will summarize our foundings and refine the next steps.**
**- We will start to write our report/data story about milestone 2 and beginning of milestone 3.**

**Dec. 12, 2017: Final Results: Based on our exploratory analysis, we will refine our data story by choosing the more meaninful results and decide on how we want to present them.**
**- We will also continue to write our report/data story.**

**Dec. 18, 2017: First complete version of the project**
**- The last day will be devoted to the finalisation of the report/data story.**

**Dec. 19, 2017 (23:59 CET): report (50%): a 4-page PDF document or a data story in a platform of your choice (e.g., a blog post, or directly in GitHub), plus the final notebook (continuation of milestone 2).**

## Questions for TAa
**- For the data that we are studying which contain geographical and temporal dimension, we can directly thing of presenting our results on a map or on a timeline or a mix of the two. Do you have other ideas or related work that we should study to find the best representation ?**

**- According to your experience, when should we start the real and final data/results interpretation ? Compared to the data exploratory, should it be longer or shorter ?**

**- The visualization of the results should be completely finished for the report ? Or the report must contain the result when the vizualisation and presentation has to be ready for the poster session ?**
